In [1]:
# This is quite critical because PyUAVCAN is asynchronous.
%autoawait

# Resetting is also necessary because you don't want to have multiple nodes running concurrently on
# the same interface -- it may not work depending on the used transport.
%reset

import logging
import pyuavcan  # READ THE LIBRARY DOCS AT http://pyuavcan.readthedocs.io
print('Using PyUAVCAN version', pyuavcan.__version__)

# In this interactive scenario there is no point generating DSDL packages programmatically because
# it is easier to invoke the DSDL code generator from the command line instead.
# For the purposes of this demo, the following command will do the job:
#
#    uvc dsdl-gen-pkg dsdl/example https://github.com/UAVCAN/public_regulated_data_types/archive/master.zip
#
# Make sure that you are invoking the command from the directory where the notebook is located.

import uavcan   # Generated package for the DSDL namespace "uavcan".
import example  # Generated package for the DSDL namespace "example".
import pyuavcan.application

IPython autoawait is `on`, and set to use `asyncio`
Once deleted, variables cannot be recovered. Proceed (y/[n])? y
Using PyUAVCAN version 0.5.0


In [2]:
# Transport configuration. Uncomment the transport you need, remove the rest.

import pyuavcan.transport.loopback
transport = pyuavcan.transport.loopback.LoopbackTransport(87)

#import pyuavcan.transport.udp
#transport = pyuavcan.transport.udp.UDPTransport('127.0.0.87/8')

#import pyuavcan.transport.serial
#transport = pyuavcan.transport.serial.SerialTransport('/dev/ttyACM0', local_node_id=87)

#import pyuavcan.transport.can
#import pyuavcan.transport.can.media.socketcan
#media = pyuavcan.transport.can.media.socketcan.SocketCANMedia('vcan0', mtu=64)
#transport = pyuavcan.transport.can.CANTransport(media, local_node_id=87)

# If you're using the loopback transport, you will see a warning about node-ID conflict, this is fine.
# We suppress it like this:
logging.getLogger('pyuavcan.application.heartbeat_publisher').setLevel(logging.ERROR)

In [3]:
import uavcan.node

# Initialize and start the node.
node_info = uavcan.node.GetInfo_1_0.Response(
    protocol_version=uavcan.node.Version_1_0(*pyuavcan.UAVCAN_SPECIFICATION_VERSION),
    software_version=uavcan.node.Version_1_0(major=1, minor=0),
    name='org.uavcan.pyuavcan.demo.jupyter',
)
node = pyuavcan.application.Node(pyuavcan.presentation.Presentation(transport), node_info)
node.start()

#import asyncio; asyncio.Task.all_tasks()  # Shows the background tasks.

In [4]:
pub_a = node.presentation.make_publisher(example.MyMessage_0_1, 1234)

sub_a = node.presentation.make_subscriber(example.MyMessage_0_1, 1234)

In [5]:
import errno
import uavcan.file

client_file_read = node.presentation.make_client_with_fixed_service_id(uavcan.file.Read_1_0, server_node_id=87)

async def file_read_handler(request: uavcan.file.Read_1_0.Request,
                            meta: pyuavcan.presentation.ServiceRequestMetadata) -> uavcan.file.Read_1_0.Response:
    print('File read request', request, 'with metadata', meta)
    try:
        with open(request.path.path.tobytes().decode(), 'r') as f:
            f.seek(request.offset)
            data = f.read(256)
        return uavcan.file.Read_1_0.Response(data=data)
    except OSError as ex:
        error_value = {
            errno.EACCES:   uavcan.file.Error_1_0.ACCESS_DENIED,
            errno.E2BIG:    uavcan.file.Error_1_0.FILE_TOO_LARGE,
            errno.EINVAL:   uavcan.file.Error_1_0.INVALID_VALUE,
            errno.EIO:      uavcan.file.Error_1_0.IO_ERROR,
            errno.EISDIR:   uavcan.file.Error_1_0.IS_DIRECTORY,
            errno.ENOENT:   uavcan.file.Error_1_0.NOT_FOUND,
            errno.ENOTSUP:  uavcan.file.Error_1_0.NOT_SUPPORTED,
            errno.ENOSPC:   uavcan.file.Error_1_0.OUT_OF_SPACE,
        }.get(ex.errno, uavcan.file.Error_1_0.UNKNOWN_ERROR)
        return uavcan.file.Read_1_0.Response(error=error_value)

server_file_read = node.presentation.get_server_with_fixed_service_id(uavcan.file.Read_1_0)
server_file_read.serve_in_background(file_read_handler)

In [6]:
await pub_a.publish(example.MyMessage_0_1(value=123.456))

message, metadata = await sub_a.receive_for(1.0)  # The timeout is in seconds.
print(message)
print(metadata)

example.MyMessage.0.1(value=123.456)
TransferFrom(timestamp=Timestamp(system_ns=1573413354064794423, monotonic_ns=465763310499644), priority=<Priority.NOMINAL: 4>, transfer_id=0, fragmented_payload=[<memory at 0x7f1f818234c8>], source_node_id=87)


In [7]:
file_read_request = uavcan.file.Read_1_0.Request(offset=2, path=uavcan.file.Path_1_0('README.md'))
response, metadata = await client_file_read.call(file_read_request)

# The type information is lost because of the serious limitations of the Python type system.
# Please upvote this issue: https://github.com/python/mypy/issues/7121
assert isinstance(response, uavcan.file.Read_1_0.Response)

first_line = response.data.tobytes().decode().split('\n')[0]
print('First line of the returned text:', repr(first_line))
print('Response metadata:', metadata)

File read request uavcan.file.Read.Request.1.0(offset=2, path=uavcan.file.Path.1.0(path='README.md')) with metadata ServiceRequestMetadata(timestamp=Timestamp(system_ns=1573413354079810420, monotonic_ns=465763325515210), priority=<Priority.NOMINAL: 4>, transfer_id=0, client_node_id=87)
First line of the returned text: 'Jupyter PyUAVCAN demo'
Response metadata: TransferFrom(timestamp=Timestamp(system_ns=1573413354081832309, monotonic_ns=465763327537201), priority=<Priority.NOMINAL: 4>, transfer_id=0, fragmented_payload=[<memory at 0x7f1f81823708>], source_node_id=87)
